In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from keras.layers import Flatten,UpSampling2D,Conv2D,Reshape,Input,Dense,Dropout,MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model,Sequential
from keras.datasets import mnist
from keras.optimizers import Adam
from keras.initializers import normal

Using TensorFlow backend.


In [17]:
randomdim=100


In [18]:
(X_train,Y_train),(X_test,Y_test)=mnist.load_data()

X_train=(X_train.astype(np.float32)-127.5)/127.5

X_train=X_train[:,:,:,np.newaxis]

In [19]:
X_train.shape

(60000, 28, 28, 1)

In [20]:
adam=Adam(lr=0.001)

In [21]:
generator=Sequential()

generator.add(Dense(128*7*7,input_dim=randomdim,kernel_initializer='normal'))

generator.add(LeakyReLU(0.25))
generator.add(Reshape((7,7,128)) )
generator.add(Conv2D(64,(5,5),padding='same'))
generator.add(MaxPooling2D(2,2))
generator.add(Dropout(0.2))
generator.add(LeakyReLU(0.25))



generator.add(Conv2D(1,(5,5),padding='same',activation='softmax',strides=2))

generator.compile(loss='binary_crossentropy', optimizer=adam)


generator.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 6272)              633472    
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 6272)              0         
_________________________________________________________________
reshape_3 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 7, 7, 64)          204864    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 3, 3, 64)          0         
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 3, 3, 64)          0         
__________

generator.summary()

In [31]:
## disciminator

disciminator = Sequential()
disciminator.add(Conv2D(64, (5,5), padding='same', strides=(2,2), kernel_initializer='normal'))
disciminator.add(LeakyReLU(0.2))
disciminator.add(Dropout(0.25))
disciminator.add(Conv2D(128, (5,5), padding='same', strides=(2,2), kernel_initializer='normal'))
disciminator.add(LeakyReLU(0.2))
disciminator.add(Dropout(0.25))
disciminator.add(Flatten())
disciminator.add(Dense(100, activation='tanh'))
disciminator.add(Dense(1, activation='softmax'))
disciminator.compile(loss='binary_crossentropy', optimizer=adam)
discriminator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 1, 1, 64)          640       
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 1, 1, 64)          0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 1, 1, 64)          0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 1, 1, 128)         73856     
_________________________________________________________________
dropout_10 (Dropout)         (None, 1, 1, 128)         0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 100)               12900     
__________

In [33]:
discriminator.trainable=False
ganinp=Input(shape=(randomdim,))

x=generator(ganinp)
ganout=discriminator(x)
gan=Model(inputs=ganinp,outputs=ganout)

gan.compile(loss='binary_crossentropy',optimizer=adam)

In [34]:
glosses=[]
dlosses=[]

In [36]:
def plotloss(epoch):
    plt.figure(figsize=(8,10))
    
    plt.plot(dlosses,label='Discriminator losss')
    plt.plot(glosses,label='generator loss')
    plt.xlabel('Epoch')
    plt.ylabel('loss')
    
    plt.legend()
    plt.savefig('images/dcgloss.png' %epoch)
  

In [44]:
def plotGeneratedImages(epoch,example=100, dim=(10,10), figsize=(10,10)):
    noise = np.random.normal(0, 1, size=[example, randomDim])
    genImages = generator.predict(noise)
    
    plt.figure(figsize=figsize)
    for i in range(genImages.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(genImages[i,0],interpolation='nearest', cmap='gray')
        plt.axis("off")
    plt.tight_layout()
    plt.savefig('images/dcgan_generated_images_%d.png' %epoch)

In [58]:
def train(epochs=1,batchsize=128):
    
    batchcount=X_train.shape[0]/batchsize
    
    print("Epochs" "%d" % epochs)
    print("batchsize" "%d" % batchsize)
    print("batchcount" "%f" % batchcount)
    
    for e in range(1,epochs+1):
        
        
        for _ in tqdm(range(batchcount)):
            
            noise=np.random.normal(0,1,size=[batchsize,randomdim])
            
            imgbatch=X_train[np.random.randint(0,X_train.shape[0],size=batchsize)]
            
            genimages=generator.predict(noise)
            
            X=np.concatenate([imgbatch,genimages])
            
            ydis=np.zeroes(2*batchsize)
            
            ydis[:batchsize]=0.9
            
            disciminator.trainable = True
            dloss = disciminator.train_on_batch(X, yDis)
            
            # train generator
            noise = np.random.normal(0, 1, size=[batchsize, randomdim])
            yGen = np.ones(batchsize)
            disciminator.trainable = False
            gloss = gan.train_on_batch(noise, yGen)
            
        dlosses.append(dloss)
        glosses.append(gloss)
        
        if e==1 or e % 50 == 0:
            plotGeneratedImages(e)
        
        plotLoss(e)
            
        
            
            



In [59]:
train(100.0,128.0)

Epochs100
batchsize128
batchcount468.750000


TypeError: 'float' object cannot be interpreted as an integer

In [1]:
Adam

Object `Adam` not found.
